In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

# Data preparation

In [5]:
df = pd.read_parquet('../../data/processed/final_data.parquet')

In [397]:
def get_lag_creator(df, **kwargs):
    for column, lags in kwargs.items():
        for lag in lags:
            df = df.assign(**{f"{column}_l{lag}": lambda _df: _df[column].shift(lag)})
    df.drop(columns = kwargs.keys(), inplace = True)
    return df

# Data prep

In [325]:
from sktime.forecasting.compose import (
    DirectTabularRegressionForecaster,
)
from sktime.forecasting.base import ForecastingHorizon

In [327]:
df["Volatility"] = (
    np.abs(np.log(df["High"]) - np.log(df["Open"]))
    * np.abs(np.log(df["High"]) - np.log(df["Close"]))
    * np.abs(np.log(df['Low']) - np.log(df['Open']))
    * np.abs(np.log(df['Low']) - np.log(df['Close']))
)

df["Volatility"] = (
    np.log(df["High"]) - np.log(df["Low"])
)

In [380]:
from sktime.transformations.series.difference import Differencer
from sktime.transformations.series.boxcox import LogTransformer
from sktime.transformations.compose import TransformerPipeline
from sktime.transformations.series.cos import CosineTransformer
from sktime.transformations.panel.compose import ColumnTransformer
from sktime.transformations.series.window_summarizer import WindowSummarizer

price_transformer = TransformerPipeline(
    [("log", LogTransformer()), ("differce", Differencer(lags=1, drop_na=True))]
)

col_transformer = ColumnTransformer(
    [
        ("price", price_transformer, ["Close", "Open", "High", "Low"]),
        ("trading", LogTransformer(), ["Volume", "Volatility"]),
        (
            "time",
            CosineTransformer(),
            ["month", "day", "dayofweek"],
        ),
        ("forum", Differencer(lags=1, drop_na=True), ["comment_count"]),
    ]
)

kwargs = {
    "target_cols": [
        "Open",
        "High",
        "Low",
        "Volume",
        "comment_count",
        "month",
        "day",
        "dayofweek",
        "Volatility",
    ],
    "lag_config": {
        "lag": ["lag", [[0, l] for l in range(7)]],
    },
}

lag_transformer = WindowSummarizer(**kwargs)

transformer_pipe = TransformerPipeline([("column", col_transformer)])
# , ("lagger", lag_transformer)

In [376]:
transformed_df = col_transformer.fit_transform(
    X=df.dropna().reset_index(drop=True)
).dropna()

In [398]:
lagged_df = get_lag_creator(
    transformed_df,
    **{
        c: [*range(1, 15)]
        for c in [
            "Open",
            "High",
            "Low",
            "Volume",
            "comment_count",
            "month",
            "day",
            "dayofweek",
            "Volatility",
        ]
    }
).dropna()

In [601]:
y = lagged_df["Close"]
X = lagged_df.drop(
    columns=["Close"]
)
# Use this if you don't want to include the forum data
# .loc[:, ~lagged_df.columns.str.startswith("comment_")]

# Forecast

In [500]:
from xgboost import XGBRegressor

xgb_regressor = XGBRegressor(
    n_estimators=150,
    max_depth=25,
    n_jobs=20,
    eval_metric=mape,
    random_state=42,
)

forecaster = DirectTabularRegressionForecaster(estimator=xgb_regressor, window_length=5)

In [602]:
from sktime.forecasting.model_selection import SlidingWindowSplitter
from sktime.forecasting.model_evaluation import evaluate
from huforpred.forecast.metrics import forecast_profit_score, rmse, mape

fh = ForecastingHorizon(values=1)
cv = SlidingWindowSplitter(fh=fh, step_length=1, window_length = 21*12, start_with_window=True)
cv_df = evaluate(
    forecaster=forecaster,
    y=y,
    X=X,
    cv=cv,
    strategy="refit",
    scoring=forecast_profit_score,
    return_data=True,
)

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

...

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

In [603]:
cv_df.to_pickle('../../data/processed/with_forum.pickle')

# Hyperparameter tuning

In [88]:
from sktime.forecasting.model_selection import ForecastingRandomizedSearchCV

In [429]:
nested_params = {
    "window_length": [5, 10, 15, 20, 25, 30],
    "estimator__max_depth": [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
    "estimator__n_estimators": [30, 50, 100, 150, 200, 250],
}

fh = ForecastingHorizon(values=1)
cv = SlidingWindowSplitter(fh=fh, step_length=12*21, window_length = 21*12, start_with_window=True)
# cv = ExpandingWindowSplitter(fh=fh, step_length=21, initial_window=21 * 9)
nrcv = ForecastingRandomizedSearchCV(
    forecaster,
    strategy="refit",
    cv=cv,
    param_distributions=nested_params,
    n_iter=1000,
    random_state=42,
    n_jobs=1,
    scoring=rmse,
    verbose=1,
)
nrcv.fit(y, fh=fh)

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 504 is smaller than n_iter=1000. Running 504 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(


Fitting 15 folds for each of 504 candidates, totalling 7560 fits


/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

...

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(
/homeEury/Bigdata/online_media/huforpred/venv/lib/python3.8/site-packages/sktime/forecasting/model_evaluation/_functions.py:143: F

ForecastingRandomizedSearchCV(cv=SlidingWindowSplitter(fh=ForecastingHorizon([1], dtype='int64', is_relative=True),
           step_length=252, window_length=252),
                              forecaster=DirectTabularRegressionForecaster(estimator=XGBRegressor(base_score=None,
                                                                                                  booster=None,
                                                                                                  callbacks=None,
                                                                                                  colsample_bylevel=None,
                                                                                                  colsample_bynode=None,
                                                                                                  colsample_bytree=None,
                                                                                                  early_stopping_rounds=N...
                                                                                                  predictor=None,
                                                                                                  random_state=None,
                                                                                                  reg_alpha=None,
                                                                                                  reg_lambda=None, ...),
                                                                           window_length=25),
                              n_iter=1000, n_jobs=1,
                              param_distributions={'estimator__max_depth': [12,
                                                                            13,
                                                                            14,
                                                                            15,
                                                                            16,
                                                                            17,
                                                                            18,
                                                                            19,
                                                                            20,
                                                                            21,
                                                                            22,
                                                                            23,
                                                                            24,
                                                                            25],
                                                   'estimator__n_estimators': [30,
                                                                               50,
                                                                               100,
                                                                               150,
                                                                               200,
                                                                               250],
                                                   'window_length': [5, 10, 15,
                                                                     20, 25,
                                                                     30]},
                              random_state=42,
                              scoring=MeanSquaredError(square_root=True),
                              verbose=1)

In [430]:
import json
Path('best_params.json').write_text(json.dumps(nrcv.best_params_))